![](../storage/banners/13_tmp.png)

### <span style="color:#0ab49a">Проверка dynamics.py.</span> <span style="color:#A254FC">Фильтр Калмана: генерация файлика</span> 

In [ ]:
from storage.observability_mapping_partial_derivatives import h_element, h_matrix

i_all = 
j_all = 
fn = 3
cn = 1
angles_navigation = False
r_f = np.array([10, 20, 30])
r_c = np.array([1, 2, 3])
q_f = None
q_с = None


H = h_matrix(i_all=i_all, j_all=j_all, fn=fn, cn=cn, angles_navigation=angles_navigation, r_f=r_f, r_c=r_c, q_f=q_f, q_с=q_с)
H